## Load the digits dataset and evolutionary_search

In [6]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
import sklearn.datasets
import numpy as np
import pandas as pd

data = sklearn.datasets.load_digits()
X = data["data"]
y = data["target"]

# make it a 2-class problem by only classifying the digit "5" vs the rest
y = np.array([1 if yy == 5 else 0 for yy in y])

X.shape, y.shape

((1797, 64), (1797,))

In [7]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

## Train an SVM with RBF kernel

### Using conventional GridSearchCV

Parameter grid: 625 parameter combinations

In [8]:
paramgrid = {"kernel": ["rbf"],
             "C"     : np.logspace(-9, 9, num=25, base=10),
             "gamma" : np.logspace(-9, 9, num=25, base=10)}
print("Size: ", len(paramgrid["kernel"])*len(paramgrid["C"])*len(paramgrid["gamma"]))

Size:  625


In [9]:
cv = GridSearchCV(estimator=SVC(),
                  param_grid=paramgrid,
                  scoring="accuracy",
                  cv=StratifiedKFold(n_splits=2),
                  verbose=1)
%time cv.fit(X, y)

Fitting 2 folds for each of 625 candidates, totalling 1250 fits
CPU times: user 3min 6s, sys: 141 ms, total: 3min 6s
Wall time: 3min 10s


[Parallel(n_jobs=1)]: Done 1250 out of 1250 | elapsed:  3.2min finished


GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-09,   5.62341e-09,   3.16228e-08,   1.77828e-07,
         1.00000e-06,   5.62341e-06,   3.16228e-05,   1.77828e-04,
         1.00000e-03,   5.62341e-03,   3.16228e-02,   1.77828e-01,
         1.00000e+00,   5.62341e+00,   3.16228e+01,   1.77828e+02,
         1.0000...7828e+05,
         1.00000e+06,   5.62341e+06,   3.16228e+07,   1.77828e+08,
         1.00000e+09])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=1)

### Best score + params

In [10]:
cv.best_score_, cv.best_params_

(0.98942682248191427, {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'})

An example of the "cannonical" cv_results_ table in sklearn:

In [34]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

,index,max_test_score,mean_test_score,min_test_score,nan_test_score?,param_index,params,std_test_score
0,1,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
254,255,0.994435,0.994435,0.994435,False,0,"{'C': 177827.941004, 'kernel': 'rbf', 'gamma':...",0.0
263,264,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
262,263,0.994435,0.994435,0.994435,False,0,"{'C': 1000000.0, 'kernel': 'rbf', 'gamma': 0.001}",0.0
261,262,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0


### Using RandomizedSearchCV

Same parameter space, but only test 250 random combinations.

In [12]:
cv = RandomizedSearchCV(estimator=SVC(),
                        param_distributions=paramgrid,
                        n_iter=250,
                        scoring="accuracy",
                        cv=StratifiedKFold(n_splits=2),
                        verbose=1)
%time cv.fit(X, y)

Fitting 2 folds for each of 250 candidates, totalling 500 fits
CPU times: user 1min 13s, sys: 93.8 ms, total: 1min 13s
Wall time: 1min 15s


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.3min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=2, random_state=None, shuffle=False),
          error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=250, n_jobs=1,
          param_distributions={'C': array([  1.00000e-09,   5.62341e-09,   3.16228e-08,   1.77828e-07,
         1.00000e-06,   5.62341e-06,   3.16228e-05,   1.77828e-04,
         1.00000e-03,   5.62341e-03,   3.16228e-02,   1.77828e-01,
         1.00000e+00,   5.62341e+00,   3.16228e+01,   1.77828e+02,
      ...7828e+05,
         1.00000e+06,   5.62341e+06,   3.16228e+07,   1.77828e+08,
         1.00000e+09])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='accuracy', verbose=1)

### Best score + params

In [13]:
cv.best_score_, cv.best_params_

(0.98942682248191427, {'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'})

An example of the "cannonical" cv_results_ table in sklearn:

In [33]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

,index,max_test_score,mean_test_score,min_test_score,nan_test_score?,param_index,params,std_test_score
0,1,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
254,255,0.994435,0.994435,0.994435,False,0,"{'C': 177827.941004, 'kernel': 'rbf', 'gamma':...",0.0
263,264,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
262,263,0.994435,0.994435,0.994435,False,0,"{'C': 1000000.0, 'kernel': 'rbf', 'gamma': 0.001}",0.0
261,262,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0


### Using EvolutionaryAlgorithmSearchCV

Again same parameter space, optimize for 10 generations.

In [15]:
cv = EvolutionaryAlgorithmSearchCV(estimator=SVC(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=10),
                                   verbose=True,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   tournament_size=3,
                                   generations_number=10)
%time cv.fit(X, y)

Types [2, 1, 2] and maxint [24, 0, 24] detected
--- Evolve in 625 possible combinations ---
gen	nevals	avg     	min    	max     
0  	50    	0.915838	0.89872	0.994435
1  	29    	0.94562 	0.89872	0.994435
2  	30    	0.989661	0.943795	0.994435
3  	29    	0.993033	0.984975	0.994435
4  	35    	0.994435	0.994435	0.994435
5  	22    	0.994435	0.994435	0.994435
6  	24    	0.990518	0.89872 	0.994435
7  	33    	0.992521	0.89872 	0.994435
8  	28    	0.990629	0.89872 	0.994435
9  	35    	0.992521	0.89872 	0.994435
10 	25    	0.994357	0.99054 	0.994435
Best individual is: {'C': 177827941.00389227, 'kernel': 'rbf', 'gamma': 0.001}
with fitness: 0.9944351697273233
CPU times: user 2min 15s, sys: 4.28 s, total: 2min 19s
Wall time: 2min 21s


### Best score + params

In [16]:
cv.best_score_, cv.best_params_

(0.99443516972732326,
 {'C': 177827941.00389227, 'gamma': 0.001, 'kernel': 'rbf'})

Our cv_results_ table (note, includes all individuals with their mean, max, min, and std test score).

In [32]:
pd.DataFrame(cv.cv_results_).sort_values("mean_test_score", ascending=False).head()

,index,max_test_score,mean_test_score,min_test_score,nan_test_score?,param_index,params,std_test_score
0,1,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
254,255,0.994435,0.994435,0.994435,False,0,"{'C': 177827.941004, 'kernel': 'rbf', 'gamma':...",0.0
263,264,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
262,263,0.994435,0.994435,0.994435,False,0,"{'C': 1000000.0, 'kernel': 'rbf', 'gamma': 0.001}",0.0
261,262,0.994435,0.994435,0.994435,False,0,"{'C': 177827941.004, 'kernel': 'rbf', 'gamma':...",0.0
